In [1]:
from urllib.parse import parse_qs

class WSGIApplication(object):

    default_headers = [
        ('Content-Type', 'text/html; charset=utf8'),
        ('Server', 'WSGIExample/1.0'),
    ]

    def __init__(self, environment, start_response_callback):
        self.environment = environment
        self.start_response = start_response_callback

    @property
    def params(self):
        params = getattr(self, '_params', None)
        if not params:
            query_string = self.environment.get('QUERY_STRING', '')
            params = {
                key: value if len(value) > 1 else value[0]
                for key, value in parse_qs(query_string).items()
            }
            setattr(self, '_params', params)
        return params

    def __iter__(self):
        self.start_response('200 OK', self.default_headers)
        greetings = 'Hello, %s!' % self.params.get('name', 'Anonymous')
        yield greetings.encode('utf-8')

In [2]:
from wsgiref.simple_server import make_server

http_server = make_server('127.0.0.1', 9090, WSGIApplication)
http_server.handle_request()

127.0.0.1 - - [18/Mar/2017 09:59:59] "GET /?name=Bob HTTP/1.1" 200 11
